In [7]:
import kfp
from kfp.components import InputPath, InputTextFile, OutputPath, OutputTextFile
from kfp.components import func_to_container_op
from typing import NamedTuple
from datetime import datetime

import sys
sys.path.insert(0, "..")

In [4]:
EXPERIMENT_NAME = "tutorial"
PIPELINE_NAME = "tutorial"
PIPELINE_VERSION = "0.0.1" # remember to change every run
PIPELINE_DESCRIPTION = "This is a tutorial pipeline"

In [13]:
@func_to_container_op
def produce_one_small_output() -> str:
    return "Hello World"

@func_to_container_op
def produce_two_small_output() -> NamedTuple('Outputs', [('text', str), ('number', int)]):
    return ('data 1', 42)

@func_to_container_op
def consume_two_arguments(text: str, nummber: int):
    print('Text={}'.format(text))
    print('Numner={}'.format(str(number)))

## Create pipeline by connecting components

In [14]:
def producers_to_consumers_pipeline(text: str="Hello world"):
    produce1_task = produce_one_small_output()
    produce2_task = produce_two_small_output()
    
    consume_task1 = consume_two_arguments(produce1_task.output, 42)
    consume_task2 = consume_two_arguments(text, produce2_task.outputs['number'])
    consume_task3 = consume_two_arguments(produce2_task.outputs['text'], produce2_task.outputs['number'])
    

## Run pipeline

In [10]:
client = kfp.Client()

In [15]:
pipeline_package_path = f"pipeline_{PIPELINE_VERSION}.yaml"
kfp.compiler.Compiler().compile(
    pipeline_func=producers_to_consumers_pipeline,
    package_path=pipeline_package_path
)

In [ ]:
#get experiment ID
experiment = 